# TMD A12B for all eta

In [59]:
!pip install -U pysr
!pip install plotly

Error in callback _flush_stdio (for post_execute):


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 4095: unexpected end of data

Julia and Julia dependencies are installed at first import:

In [61]:
import pysr

Error in callback _flush_stdio (for post_execute):


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 4095: unexpected end of data

Now, let's import everything else as well as the PySRRegressor:


In [63]:
import sympy
import numpy as np
from matplotlib import pyplot as plt
from pysr import PySRRegressor
from sklearn.model_selection import train_test_split
import h5py

Error in callback _flush_stdio (for post_execute):


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 4095: unexpected end of data

# PySR for A12B for all eta:


In [65]:

file_path = "/Users/hariprashadravikumar/sivers_TMD_PhD_project/save_h5_A12B_A2B/eta_bL_bT_A12B_err_wt_NEnergyJakkif.h5"
data_list = []
with h5py.File(file_path, "r") as h5_file:
    for eta in range(7, 11):
        dataset_name = f"eta_{eta}_bL_bT_A12B_err"
        data_list.append(np.array(h5_file[dataset_name]))
        
data_for_eta = np.vstack(data_list)
    
# Extract bLbT and A12B
etabLbT = data_for_eta[:, 0:3]  
A12B = data_for_eta[:, 3]   
A12Berr = data_for_eta[:, 4]
A12Bweight = [1/sigma**2 for sigma in A12Berr]


#print("etabLbT =", etabLbT)
#print("A12B =", A12B)
#print("A12Berr =", A12Berr)

Error in callback _flush_stdio (for post_execute):


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 4095: unexpected end of data

In [66]:
import plotly.graph_objects as go

# Extract unique values for bT
unique_bT = np.unique(etabLbT[:, 2])


fig = go.Figure()
for i, bT in enumerate(unique_bT):
    # Filter data
    mask = etabLbT[:, 2] == bT
    eta_values = etabLbT[mask, 0]
    bL_values = etabLbT[mask, 1]
    A12B_values = A12B[mask]
    A12B_errors = A12Berr[mask]

    # Add trace for each bT
    fig.add_trace(go.Scatter3d(
        x=eta_values , y=bL_values, z=A12B_values,
        mode="markers",
        error_z=dict(type="data", array=A12B_errors, visible=True),
        marker=dict(size=6),
        name=f"bT = {int(bT)}"
    ))

# Update layout
fig.update_layout(
    scene=dict(
        xaxis_title="eta",
        yaxis_title="bL",
        zaxis_title="A12B",
    ),
    title="Plot of A12B vs bL & eta for Different bT Values",
    margin=dict(l=0, r=0, b=0, t=40),
)

# Show interactive plot
fig.show()


Error in callback _flush_stdio (for post_execute):


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 4095: unexpected end of data

By default, we will set up 30 populations of expressions (which evolve independently except for migrations), use 4 threads, and use `"best"` for our model selection strategy:

In [68]:
default_pysr_params = dict(
    populations=50,
    model_selection="best",
)

Error in callback _flush_stdio (for post_execute):


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 4095: unexpected end of data

In [69]:
# test try functions
x = np.linspace(-10, 10, 1000)
y = 1 / (np.sqrt(1+x**2))

#plt.plot(x, y)
#plt.show()

Error in callback _flush_stdio (for post_execute):


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 4095: unexpected end of data

PySR can run for arbitrarily long, and continue to find more and more accurate expressions. You can set the total number of cycles of evolution with `niterations`, although there are also a [few more ways](https://github.com/MilesCranmer/PySR/pull/134) to stop execution.

**This first execution will take a bit longer to startup, as the library is JIT-compiled. The next execution will be much faster.**

In [71]:
# Learn equations
from sympy import exp, symbols, Function

# Update the PySRRegressor call
model = PySRRegressor(
    niterations=30,
    binary_operators=["*", "Lorentzian(x, y) = 1/(y+x^2)"],
    unary_operators=["expdecay(x) = exp(-x)", "gaussian(x) = exp(-x^2)"],
    extra_sympy_mappings={"expdecay": lambda x: exp(-x), "gaussian": lambda x: exp(-x**2), "Lorentzian": lambda x, y: 1/(y+x**2)},
    constraints={'^': (-1, 1)}
)
model.fit(etabLbT, A12B, weights = A12Bweight, variable_names = ['n','bL', 'bT']) 

/opt/homebrew/anaconda3/lib/python3.11/site-packages/pysr/sr.py:2766: UserWarning:

Note: it looks like you are running in Jupyter. The progress bar will be turned off.

[ Info: Started!
[ Info: Final population:
[ Info: Results saved to:


PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                     -0.00015257795   
	1         0.008822                        gaussian(bT) * -0.004544321   
	2         0.276181       Lorentzian(bL, -0.88513273) * -0.00031499477   
	3         0.244979             Lorentzian(bL, bT * bT) * -0.008327655   
	4         0.210081  Lorentzian(bL, 1.3305786) * (Lorentzian(bT, 0....   
	5         0.076198  Lorentzian(bL, n) * (Lorentzian(bL, bT * bT) *...   
	6   >>>>  0.168613  Lorentzian(n, -5.7606587) * ((Lorentzian(bL, 1...   
	7         0.094710  (-2.0755036 * (Lorentzian(bT, 0.5699667) * Lor...   
	8         0.093704  ((Lorentzian(n, -3.2981837) * Lorentzian(bT, 0...   
	9         0.000534  (Lorentzian(n, -3.2981837) * (Lorentzian(bT, e...   
	10        0.007226  Lorentzian(n, -3.2981837) * (Lorentzian(bL, Lo...   
	11        0.011043  Lorentzian(bL, Lorentzian(bL, 0.44178534)) * (...   
	12        0.001667  Lorentzian(n, 1.3644105 * Lorentzian(Lorentzia...   
	13        0.003510  Lorentzian(bL, Lorentzian(bL, 0.44178534)) * (...   
	14        0.027785  Lorentzian(bT, gaussian(bT)) * ((Lorentzian(n,...   
	15        0.015955  (Lorentzian(n, Lorentzian(Lorentzian(Lorentzia...   
	16        0.020492  (Lorentzian(n, Lorentzian(Lorentzian(Lorentzia...   
	17        0.010294  -0.98116434 * (Lorentzian(bL, bT * bT) * Loren...   
	
	            loss  complexity  
	0   1.757340e-07           1  
	1   1.711438e-07           4  
	2   1.298426e-07           5  
	3   7.954839e-08           7  
	4   5.225859e-08           9  
	5   4.487175e-08          11  
	6   3.202710e-08          13  
	7   2.913303e-08          14  
	8   2.652714e-08          15  
	9   2.651298e-08          16  
	10  2.632210e-08          17  
	11  2.603304e-08          18  
	12  2.594639e-08          20  
	13  2.576488e-08          22  
	14  2.437217e-08          24  
	15  2.360673e-08          26  
	16  2.265878e-08          28  
	17  2.219705e-08          30  
]

Error in callback _flush_stdio (for post_execute):


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 4095: unexpected end of data

We can print the model, which will print out all the discovered expressions:

We can also view the SymPy format of the best expression:

In [74]:
model.sympy()

-1.65816520000000/((n**2 - 5.7606587)*(bL**2 + 1.2830195)*(bT**2 + 0.40749484))

Error in callback _flush_stdio (for post_execute):


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 4095: unexpected end of data

We can also view the SymPy of any other expression in the list, using the index of it in `model.equations_`.

In [76]:
model.equations_[["complexity", "loss", "equation"]]

,complexity,loss,equation
0,1,1.757340e-07,-0.00015257795
1,4,1.711438e-07,gaussian(bT) * -0.004544321
2,5,1.298426e-07,"Lorentzian(bL, -0.88513273) * -0.00031499477"
3,7,7.954839e-08,"Lorentzian(bL, bT * bT) * -0.008327655"
4,9,5.225859e-08,"Lorentzian(bL, 1.3305786) * (Lorentzian(bT, 0...."
5,11,4.487175e-08,"Lorentzian(bL, n) * (Lorentzian(bL, bT * bT) *..."
6,13,3.202710e-08,"Lorentzian(n, -5.7606587) * ((Lorentzian(bL, 1..."
7,14,2.913303e-08,"(-2.0755036 * (Lorentzian(bT, 0.5699667) * Lor..."
8,15,2.652714e-08,"((Lorentzian(n, -3.2981837) * Lorentzian(bT, 0..."
9,16,2.651298e-08,"(Lorentzian(n, -3.2981837) * (Lorentzian(bT, e..."


Error in callback _flush_stdio (for post_execute):


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 4095: unexpected end of data